In [2]:
import json
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

## Endpoints principais

In [3]:
# Endpoints
urls = {
    'deputados':'https://dadosabertos.camara.leg.br/api/v2/deputados',
    'proposicoes': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes',
    'detalhe_proposicao':'https://dadosabertos.camara.leg.br/api/v2/proposicoes/%s'
}

# Avaliable payloads
available_payloads = {
     'deputados': [ 
                    'idLegislatura',
                    'siglaUf',
                    'siglaPartido',
                    'siglaSexo',
                    'pagina',
                    'itens',
                    'ordem',
                    'ordenarPor',
                    'id'
    ],
    
    'proposicoes':[
                    'id',
                    'ano',
                    'itens',
                    'autor'
    ],
}

## Funções para facilitar o fetch e o tratamento de dados

* Aqui começa a exploração dos dados, ainda de forma bem simples, com o objetivo de:

    * Testar os endpoints
    * Identificar os dados que são trazidos
    * Identificar o formato e estrutura dos dados de resposta

In [4]:
def get_data_by(data_type, payload=None):
    if payload != None:
        for column in payload.keys():
            if column not in available_payloads[data_type]:
                print('Column %s not available!' % column)
                return None
        
    response = requests.get(urls[data_type], params=payload)
    if response.status_code == 200:
        return json_to_dataframe(response.json()['dados'], 'records')
#             return response.json()['dados']
    
    return None


def json_to_dataframe(data_json, orient):
    return pd.read_json(json.dumps(data_json), orient=orient)


def get_details_data(data_type, identifier):
    response = requests.get(urls[data_type]%identifier)
    return response.json()['dados']


In [5]:
params = {
    'id': '178950',
}

deputados = get_data_by('deputados', params)
deputados

,id,idLegislatura,nome,siglaPartido,siglaUf,uri,uriPartido,urlFoto
0,178950,55,ANTÔNIO JÁCOME,PODE,RN,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...


In [6]:
params = {
    'id': '2120097',
#     'autor': {
#         'siglaUF':'12'
#     }
}

proposicoes = get_data_by('proposicoes', params)
proposicoes

,ano,ementa,id,idTipo,numero,siglaTipo,uri
0,2017,Requer a realização de audiência pública para ...,2120097,147,19,REQ,https://dadosabertos.camara.leg.br/api/v2/prop...


* Não é possível converter direto pra o dataframe pois é necessário antes tratar o json

In [7]:
proposicao = get_details_data('detalhe_proposicao', '2120097')
proposicao[u'situacaoProposicao'] = proposicao['statusProposicao']['descricaoSituacao']
proposicao[u'descricaoTramitacao'] = proposicao['statusProposicao']['descricaoTramitacao']
del proposicao['statusProposicao']

In [8]:
proposicao_df = pd.DataFrame([proposicao], columns=proposicao.keys())
proposicao_df.columns

Index([u'situacaoProposicao', u'ano', u'texto', u'idTipoAutor', u'keywords',
       u'uriUltimoRelator', u'idTipo', u'id', u'uriPropPosterior',
       u'uriPropPrincipal', u'ementa', u'justificativa', u'siglaTipo',
       u'dataApresentacao', u'numero', u'uriOrgaoNumerador', u'urnFinal',
       u'uriAutores', u'ementaDetalhada', u'uriPropAnterior', u'tipoAutor',
       u'uri', u'urlInteiroTeor', u'descricaoTramitacao', u'descricaoTipo'],
      dtype='object')